### Example of Genuine Multipartite Non-Locality Test In A Quantum Processor Using Braket

In this notebook we explain how to test Svetlichny and Mermin inequalities in a quantum processor. For that goal, we use the Braket SDK to produce some states and the associated circuits needed to test nonlocality inequalities in both, local simulators or real quantum processors. These developments are based on the research presented in [Granda Arango, A. et al, Physical Review A, in press, 2025; https://journals.aps.org/pra/accepted/10.1103/8nbw-bvj2 ; arXiv:2405.01650v3 [quant-ph]].

This notebook is organized following the setps below. hola

(1) First, we show how to create quantum states. We start with the GHZ states, and then provide a function for creating quantum random circuits.

(2) Next, we build all the functions needed for the formulation of the nonlocality inequalities studied here (Mermin and Svetlichny). Given an input circuit producing the quantum state $\rho$, using these tools one can compute the angles of maximal violation of Svetlichny (or Mermin) inequality and, with those angles, build the circuits that emulate the local spin measurements used to compute the corresponding correlation terms.

(3) Once the numerical calcultation in (1) and (2) are done, we are ready to implement the inequaltiies tests in local simulators or actual QPUs. Here we show how to do the evaluation in noiselss and noisy simulators, and briefly indicate how to run the circuits in an actual QPU.

In [ ]:
# general imports
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline    
import time
import numpy as np
# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Observable, Gate
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
from braket.circuits.noise_model import (GateCriteria, NoiseModel,
                                         ObservableCriteria)
from braket.circuits.noises import (AmplitudeDamping, BitFlip, Depolarizing,
                                    PauliChannel, PhaseDamping, PhaseFlip,
                                    TwoQubitDepolarizing)

### (1) Generation of different types of quantum states

We need to prepare suitable quantum states to analyze the simulation of nonlocality inequalities.

Let us first create a three qubits GHZ state: $$|\psi\rangle = \frac{1}{\sqrt{2}}(|000\rangle + |111\rangle)$$

In [ ]:
GHZ = Circuit()
GHZ.h(0)
GHZ.cnot(0,1)
GHZ.cnot(1,2)

We can define a function for creating their generalization to N qubits:
$$|\psi_N\rangle = \frac{1}{\sqrt{2}}(|00\ldots 0\rangle + |11\ldots 1\rangle)$$




In [ ]:
### This function creates a GHZ of N qubits, and extracts the associated density matrix:

def ghz_circuit(N): 
    ### Create circuit preparing a GHZ state of N qubits
    GHZ_circuit = Circuit()
    GHZ_circuit.h(0)
    for j in range(0,N-1):
        GHZ_circuit.cnot(j,j+1)   
    ### Extract density matrix from circuit     
    GHZ_circuit.density_matrix(target=range(N))   
    device = LocalSimulator(backend="braket_dm")
    task = device.run(GHZ_circuit, shots=0)
    result = task.result()
    RHO = result.values[0]    
    return GHZ_circuit, RHO

Let us now produce some quantum random circuits invoking the function QRC. What QRC does is, on each layer, chose a random family of gates out of given set of elementary gates. After several steps, one obtains a random unitary which approximates the Haar measure (see [Granda Arango, A. et al, Physical Review A, in press, 2025; https://journals.aps.org/pra/accepted/10.1103/8nbw-bvj2 ; arXiv:2405.01650v3 [quant-ph]]).

In [ ]:
from QRC import QRC

### Now we produce a list of states for testing the Svetlichny inequality on them. 
### Start with an empty list:

N = 3

states = []

### Add a GHZ state circuit together with its associated density matrix:

states.append([ghz_circuit(N)[1], ghz_circuit(N)[0]])

### Add some quantum random circuits, using the QRC function contained in QRC_parallel.py:

for i in range(10):
    a = QRC(N=N,D=10, max_gates=2)
    states.append([a["Ideal RHO"], a["Ideal Circuit"]])   

### (2) Building nonlocality inequalities main functions

Here we build the functions for evaluating Svetlichny and Mermin inequalities, together with the tools to build a set of quantum circuits for testing them in QPUs.

### (2.1) Svetlichny inequality

Here start by showing the functions needed for formulating the Svetlichny inequality. We follow reference M. Seevinck and G. Svetlichny, Phys. Rev. Lett. 89, 060401, 2002 https://doi.org/10.1103/PhysRevLett.89.060401 , as it allows to address an arbitrary number of qubits N.


In [ ]:
########## Defining observables and optimization functions #############################

############################################################
### Some functions needed to define 1 qubit rotations ####
############################################################

### We will use local rotations. Thus, we need the matrices associated to different types 
### of qubits rotations.

sigma_z = np.array([[1, 0], [0, -1]])

### Rotation in z

def theoretical_Rz(theta):
    R = np.array([[np.exp(-1j * theta/2) , 0], [0 , np.exp(1j * theta/2)]])
    return R

### Rotation in x

def theoretical_Rx(theta):
    R = np.array([[np.cos(theta/2), (-1j)*np.sin(theta/2)],
        [(-1j)*np.sin(theta/2), np.cos(theta / 2)]])
    return R

### R gate:

def theoretical_R(theta, phi):
    cos_theta_over_2 = np.cos(theta / 2)
    sin_theta_over_2 = np.sin(theta / 2)
    R = np.array([
        [cos_theta_over_2, -1j * np.exp(-1j * phi) * sin_theta_over_2],
        [-1j * np.exp(1j * phi) * sin_theta_over_2, cos_theta_over_2]
    ])
    return R

def R_sigma_z_Rdagger(theta, phi):
    R = theoretical_R(theta, phi)
    R_dagger = R.conj().T
    return R @ sigma_z @ R_dagger

# Computing R_dagger * sigma_z * R
def R_dagger_sigma_z_R(theta, phi):
    R = theoretical_R(theta, phi)
    R_dagger = R.conj().T
    return R_dagger @ sigma_z @ R

############################################################
### What follows is needed fot he Svetlichny inequality ####
############################################################

### Equation (4) in PRL 89, 060401:

def nu(k, branch = False):
    nu0 = (-1)**(k*(k+1)/2)
    return nu0

### This function returns t in equation (5) of PRL 89, 060401:

def sign(t):
    s = t.count(1)
    return s

### These functions are needed to do tensor products of multiple matrices and vectors:

import itertools

def KroneckerTwoSets(L1, L2):
    P = []
    for i in range(len(L1)):
        for j in range(len(L2)):
            P.append(np.kron(L1[i], L2[j]))
    return P

def KroneckerListOfLists(L):
    result = L[0]
    for x in L:
        result = KroneckerTwoSets(result, x)
    return result

def KroneckerMultiplyList(myList):
    # Multiply elements one by one
    result = [[1]]
    for x in myList:
        result = np.kron(result, x)
    return result

### This function gives the maximal violation angles for GHZ states of 
### N qubits (see discussion about GHZ states in page 3 PRL 89, 060401):

def angles(N):
    angles1 = np.zeros(N)
    angles1[0] = np.pi/4
    angles2 = np.zeros(N)
    angles2[0] = np.pi/4 + np.pi/2
    for i in range(1,N):
        angles2[i] = np.pi/2
    L = []    
    for i in range(N):
          L.append([angles1[i],angles2[i]])      
    return L, angles1, angles2

###############################################################################
### This function does the optimization of angles in Svetlichny inequality ####
###############################################################################

### What the following function does is, given an arbitrary quantum state, it computes the orientations of local spin measurements needed for 
### obtaining maximal violation of Equation (6) of PRL 89, 060401.

def SVL(rho,N,angles):
    OPS = []
    l = [0, 1]
    M = list(itertools.product(l, repeat=N))
    for A in range(N):
        theta1 = angles[A][0]
        phi1 = angles[A][1]
        theta2 = angles[A][2]
        phi2 = angles[A][3]
        # A1 =  theoretical_R(theta1,phi1) @ sigma_z @ theoretical_R(theta1,phi1).conj().T 
        # A2 =  theoretical_R(theta2,phi2) @ sigma_z @ theoretical_R(theta2,phi2).conj().T 
        A1 =  (theoretical_Rz(phi1)) @ (theoretical_Rx(theta1)) @ sigma_z @ (theoretical_Rx(theta1).conj().T) @ (theoretical_Rz(phi1).conj().T) 
        A2 =  (theoretical_Rz(phi2)) @ (theoretical_Rx(theta2))@ sigma_z @ (theoretical_Rx(theta2).conj().T) @ (theoretical_Rz(phi2).conj().T) 
        OPS.append([A1,A2])
    Terms = []  
    for I in M:
        Term = []
        for j in range(len(I)):
            a = OPS[j][I[j]]
            Term.append(a)
        T = nu(sign(I))*KroneckerMultiplyList(Term)
        Terms.append(T)
    S = sum( t for t in Terms )
    Svl= np.sqrt((np.trace(S@rho).astype(float))**2)
    return -Svl       

##############################################################################################
##### This function implements the circuits needed for spin observables for Svetlichny #######
##############################################################################################

# Once the angles of maximal violation are computed (as explained in https://arxiv.org/abs/2405.01650), one needs the circuits based on local rotations 
# of qubits in order to emulate those measurements. This is what the function SVL_circuits does.
# The output is a list with the information needed for implementing the circuits that emulate the local 
# sin measurements of each correlation term, together with the sign it has in the inequality.

def SVL_circuits(circuit,N,angles):
    C_M = []
    l = [0, 1]
    M = list(itertools.product(l, repeat=N))
    for A in range(N):
        theta1 = angles[A][0]
        phi1 = angles[A][1]
        theta2 = angles[A][2]
        phi2 = angles[A][3]
        M1 = Circuit()          
        M1.rz([A], -phi1) 
        M1.rx([A], -theta1) 
        M2 = Circuit()
        M2.rz([A], -phi2)
        M2.rx([A], -theta2)
        C_M.append([M1,M2]) 
    signed_circuits = []  
    for I in M:
        correlation_circuit = Circuit()
        circuit_copy = Circuit()
        c = circuit_copy.add(circuit.instructions)
        l = []
        for j in range(len(I)):
            l.append(C_M[j][I[j]]) 
        result_circuit = correlation_circuit
        for element in l:
            result_circuit = result_circuit.add_circuit(element)
        d = c.add_circuit(result_circuit)        
        s = nu(sign(I))
        ### Here we store the circuit "d" associated to a correlation term, 
        ### together with the sign "s" this correlation term has associated in the Svetlichny inequality. 
        signed_circuits.append([d,s])
    return signed_circuits       


### (2.2) Mermin

Here we show the functions needed for Mermin inequalities.

In [ ]:
###############################################################################
##### Now we build the main functions for Mermin polinomials ##################
###############################################################################

import itertools

# Define first Mermin polinomial:
M_0 = [[1,[0]]]

# Define iterative step in Mermin function

def Mermin_step(M):
    key_func = lambda x: x[1] 
    F0 = []
    F1 = []
    F2 = []
    F3 = []
    M_primed = []
    for x in M:
        t = []
        for j in range(len(x[1])):
            if x[1][j] == 0:
                t.append(1)
            else:
                t.append(0)    
        E = [x[0],t]
        M_primed.append(E)    
    for i in range(len(M)):
        T0 = [M[i][0],M[i][1]+[0]]
        T1 = [M[i][0],M[i][1]+[1]]
        T2 = [M_primed[i][0],M_primed[i][1]+[0]]
        T3 = [M_primed[i][0]*(-1),M_primed[i][1]+[1]] 
        F0.append(T0)
        F1.append(T1)
        F2.append(T2)
        F3.append(T3)
    M_step_0 = F0 + F1 + F2 + F3
    M_step = []
    for key, group in itertools.groupby(sorted(M_step_0, key=lambda x: x[1]), lambda x: x[1]):
        l = []
        for x in group:
            l.append(x[0])     
        s = sum(l)    
        if s != 0:
            T = [s, key] 
            M_step.append(T)
    return M_step

# Now we can define the Mermin choice of observables for arbitrary N. 

def Mermin_choice(N):
   L = M_0
   steps = N-1
   for _ in range(steps):
      L = Mermin_step(L)
   return L   

# Mermin optimization function

def Mermin(rho, N, angles):
    OPS = []
    # Call the Mermin choice of observables with their coefficients in the inequality. 
    M = Mermin_choice(N)
    for A in range(N):
        theta1 = angles[A][0]
        phi1 = angles[A][1]
        theta2 = angles[A][2]
        phi2 = angles[A][3]
        # A1 =  theoretical_R(theta1,phi1) @ sigma_z @ theoretical_R(theta1,phi1).conj().T 
        # A2 =  theoretical_R(theta2,phi2) @ sigma_z @ theoretical_R(theta2,phi2).conj().T 
        A1 =  (theoretical_Rz(phi1)) @ (theoretical_Rx(theta1)) @ sigma_z @ (theoretical_Rx(theta1).conj().T) @ (theoretical_Rz(phi1).conj().T) 
        A2 =  (theoretical_Rz(phi2)) @ (theoretical_Rx(theta2))@ sigma_z @ (theoretical_Rx(theta2).conj().T) @ (theoretical_Rz(phi2).conj().T) 
        OPS.append([A1,A2])
    Terms = []  
    for I in M:
        Term = []
        for j in range(len(I[1])):
            a = OPS[j][I[1][j]]
            Term.append(a)
        T = I[0] * KroneckerMultiplyList(Term)
        Terms.append(T)
    M_operator = sum( t for t in Terms )
    violation = np.sqrt((np.trace(M_operator@rho).astype(float))**2)
    return -violation

# Function needed to transform maximal violation angles into circuits for testing Mermin inequality in a QPU

def Mermin_circuits(circuit,N,angles):
    C_M = []
    M = Mermin_choice(N)
    for A in range(N):
        theta1 = angles[A][0]
        phi1 = angles[A][1]
        theta2 = angles[A][2]
        phi2 = angles[A][3]
        M1 = Circuit()          
        M1.rz([A], -phi1) 
        M1.rx([A], -theta1) 
        M2 = Circuit()
        M2.rz([A], -phi2)
        M2.rx([A], -theta2)
        C_M.append([M1,M2]) 
    signed_circuits = []  
    for I in M: 
        correlation_circuit = Circuit(N)
        c = circuit.copy()
        l = []
        for j in range(len(I[1])):
            l.append(C_M[j][I[1][j]])   
        result_circuit = correlation_circuit
        for element in l:
            result_circuit = result_circuit.add_circuit(element)
        d = c.add_circuit(result_circuit)  
        coefficient = I[0]
        signed_circuits.append([d,coefficient])
    return  signed_circuits  

### (2.3) Optimization problem

Given a quantum state $\rho$ and the Svetlichny inequality, the evaluation of the modulus of the mean value of $\rho$ on the Svetlichny operator depends on the angles of the local spin measurements (see discussion in [Granda Arango, A. et al, Physical Review A, in press, 2025; https://journals.aps.org/pra/accepted/10.1103/8nbw-bvj2 ; arXiv:2405.01650v3 [quant-ph]]). Thus, for a given state, we compute the angles of maximal violation in what follows. These angles, together with the corresponding circuit preparing the state to be tested, will be the inputs of the functions SVL_circuits and Mermin_circuits, which produce the circuits used to measure the correlations appearing in the corresponding inequality. We do this for each state of the list defined in (2.1).

In [ ]:
### Number of iterations in the optimization process. It is crucial when the number of qubits increases.
### As the number of qubits grow, more iterations are needed. For three qubits we recommend using seeds = 20.
### But you can use a shorter number if you need quick results.

### This part of the code can be paralelized easily. Do it if you work wih a server.

from random import random
import warnings
from scipy import optimize
from operator import itemgetter

N = 3

seeds= 10

#### Task of computing the maximum Svetlichny violation angles for a given RHO as input.

def task(RHO):
    warnings.filterwarnings('ignore')
    def f(params):
        angs = [params[4 * i:4 * (i + 1)] for i in range(N)]
        return SVL(RHO[0],N,angs)       
    partial_results = []
    done_seeds = 0
    while done_seeds < seeds:
    #for _ in range(seeds):
        initial_guess = [] 
        for _ in range(4*N):
            y =  2*(np.pi)*(random())
            initial_guess.append(y)
        bnds = list(itertools.repeat((0, 2*np.pi),4*N))    
        res = optimize.minimize(f, initial_guess, bounds = bnds)
        if res.success:
            fitted_params = res.x
            if np.all(fitted_params):
                done_seeds += 1
                partial_results.append([-f(res.x),fitted_params,RHO[1]])  

    value = max(partial_results, key=itemgetter(0))
    return value

In [ ]:
#############################################
########### Optimizer for N qubits ##########
#############################################
    
#### Now we enter the optimization.

from random import random
from time import sleep
import multiprocessing as mp
from multiprocessing import Pool
import time


### Lists where we store the circuits, parmeters obtained, and the violation values.
### There will be a concrete circuit (to be stored in the list "circuits") for each
### state contained in the list "states" created above.
### The lists "parameters" and "values" will contain the angles and value of maximal violation, respectively, 
### for each state contained in "states". Notice that all the lists "states", "circuits", "parameters" and "values"
### have the same lenght. Also, the order in which the items are presented is the same as that of "states". For
### example, "states[10]" is the tenth quantum state produced above, "circuits[10]" is the circuit needed to produce it
### in a quantum computer or simulator, "parameters[10]" are the angles needed for obtaining a Maximal violation of the
### Svetlichny inequality, and "values[10]" are the actual (ideal) violation value that one would obtain in a noiseless quantum computer.

circuits = []
parameters = []
values = []

def pool_handler():
    p = Pool()
    result = p.map_async(task,states) 
    while not result.ready():
        print("num left: {}".format(result._number_left))
        time.sleep(1) 
    values = [x[0] for x in result.get()] 
    parameters = [x[1]  for x in result.get()]
    circuits = [x[2]  for x in result.get()]
    p.close()
    p.join()
    violations = []
    for x in values:
        if x > 2**(N-1):
            violations.append(x)
    print("Fraction: ", len(violations)/len(values)) 
    return values, parameters, circuits

if __name__ == '__main__':
    values, parameters, circuits = pool_handler() 

# (2.4) Create a list of circuits to test nonlocality inequality

We now use the function SVL_circuits to build, for each state $\rho$, the circuits that implement the Svetlichny inequality. A similar procedure can be followed to test Mermin's inequality using Mermin_circuits.

In [ ]:
#### Here we create the circuits that will be run in a QPU backend. Each state in the list "states" will give place to a
#### different Svetlichny test (or Mermin test, in case you are doing that). Each Svetlichny test associated to a state taken from "states"
#### will be implemented as a list of circuits (one for each correlation term appearing on the corresponding inequality). 
#### All the tests produced will be sotred in the list "circuits_for_backends" (notice that the lenght of this list will be the same as that of "states").
#### For each test, the information needed to implement it on a quantum computer or simulator will be stored in a dictionary object ("experiment_dict" below).
#### 

import pickle 

circuits_for_backends =  []

import os
if not os.path.exists('circuits_for_experiments'):
   os.makedirs('circuits_for_experiments')   

###

for j in range(len(circuits)):
    #### Notice that information will be stored in a dictionary object
    #### This is done for each element of the chosen list, so if your list
    #### had, say, ten states/circuits, then, you will have ten instances of
    #### Svetlichny inequality experiments (i.e., ten dictionaries).
    experiment_dict = {}
    angles = [parameters[j][4 * i:4 * (i + 1)] for i in range(N)] 
    c = circuits[j]
    value = values[j]
    circuits_for_test = SVL_circuits(c,N,angles)
    ### We store the violation value
    experiment_dict["Theoretical_violation"] = value
    ### We store the circuits together with the sign their correlations have in the SVL inequality.
    ### As an example, for three qubits you will have eight circuits, for four sixteen, and so on.
    ### Notice that each element of circuits_for_test is a list with two elements: a quantum circuit
    ### (associated to a correlation term in the inequality), together with the sign this term carries in
    ### the inequality.
    experiment_dict["Signed_circuits"] = circuits_for_test
    circuits_for_backends.append(experiment_dict)
    ### At this point, it is better to save each test generated in some folder,
    ### so you can use it possibly in combination with other codes.
    with open('circuits_for_experiments/experiment_dict_' + str(j) + '.pkl', 'wb') as f:
        pickle.dump(experiment_dict, f)
        
### We recommend to open one example of the circuits produced to see how information is structured.
### You will need to understand this if you want to do your own developments and postprocess data.

with open('circuits_for_experiments/experiment_dict_0.pkl', 'rb') as f:
    data_for_experiment = pickle.load(f)    

print(data_for_experiment)    

### (3) Run circuits on Local Simulators

Now that we have the circuits needed to perform a Svetlichny test (or Mermin, if it is the case) for each state in the list "states", 
let us start with an example on a local (noiseless) simulator.

For the case of three qubits, there are eight correlation terms in the Svetlichny inequality. Each of these terms is formed by a circuit $C_s$ implementing the state, say, a GHZ state, concatenated with a circuit $C_l$ implementing the local spin measurements. Recall that the angles are set to idealy provide the maximal violation of the Svetlichny inequality.

### (3.1) Noiseless simulation

Here we explain how to run the Svetlichny inequality in a noiseless simulator. A Mermin test works in a completely analogous way. 

We first need to define a function that, given a concrete nonlocality test (as the elements of the list "circuits_for_backends" are), performs the following tasks:

(1) For each circuit associated to a correlation term, run it in a QPU or simulator, and extract the counts. Out of them, compute the corresponding mean values.

(2) Use the results obtain in (1) to compute the left hand side of the Svetlichny (or Mermin) inequality. Recall that such a quantity is a sum of correlation terms.

This is what the function "run_test()" below does. Notice that it depends on how data is structured in the input dictionary ("test"). So, if you modify it for your purposes, take this fact into account. You can modify this function mutatis mutandis to adapt it to postprocess the data you extract from running your circuits on an actual QPU (but concrete implementation will depend on how you structure your data).

In [ ]:
def run_test(test,device,shots, noise = "False"):
        ### Define empty list to store the correlations of each term of the inequality. 
        correlations = []
        ### We iterate over all the circuits associated to the test of a given state
        for i in range(len(test['Signed_circuits'])):
            ### For three qubits, results are of the form 000, 001, 011, etc., which, in terms of spin, 
            ### read as (1)(1)(1)=1, (1)(1)(-1)=1, (1)(-1)(-1)=1 etc. We must take this into account and compute 
            ### the counts of 1 and -1 when calculating the mean values for the products of operators. 
            ### Each time we obtain a 1 it adds to the probability p_plus, and -1 contributes to the probability
            ### of p_minus. We start by setting both quantities equal to zero. For N qubits one can proceed in an analogous way.
            p_plus = 0 
            p_minus = 0 
            ### Here we produce a clone of each circuit in the test:
            input_circuit = test['Signed_circuits'][i][0].copy()
            ### In anoisy simulation we set noise == "True" to add the noise model to the ideal circuit. 
            if noise == "True":
                 circuit = noise_model.apply(input_circuit)
            else:
                 circuit = input_circuit
            ### This will be used to run the circuit in a simulator.     
            task = device.run(circuit, shots) # Run circuit.
            result = task.result() # Extract the result.
            counts = result.measurement_counts # Extract counts.
            for key in counts: # Iterate over all possible counts.
                count = key.count("1") # See how many times "1" appears in a given count.
                if (-1)**(count) < 0: # If it appears an odd number of times, the output of the product of observables will be (-1), thus contributing to p_minus.
                    p_minus += counts[key]/S
                else: # If it appears an even number of times, the output of the product of observables will be (1), thus contributing to p_plus.
                    p_plus += counts[key]/S  
                pre_term =  p_plus - p_minus # This is the mean value for the product of the local spin observables: p_plus*(1) + p_minus(-1). 
            term = (test['Signed_circuits'][i][1])*pre_term # Now you must multiply by the coefficient the above mean value carries in the corresponding inequality.
            correlations.append(term) # Store in the correlation terms list.
            del(circuit)    
        value = test["Theoretical_violation"] # Extract the ideal value, to compare.
        mean = np.abs(sum(correlations)) # Now sum up everything and take the moduluis, to obtain the left-hand side of the inequality. 
        print("Obtained vs ideal value:", mean, value) # Display results.
        return mean, value # Return relevant data.

In [ ]:
### Each element x of the list of circuits contains the information for testing the Svetlichny (or Mermin) inequality
### in a QPU or simulator.

### Define number of shots:

S = 1000

### Define simulator:

device = LocalSimulator()

### Now, for each element of the list "circuits_for_backends", we sample over all the 
### corresponding circuits, and compute the modulus of the mean value of the state over 
### the Svetlichny operator. Recall that each element of "circuits_for_backends" corresponds 
### to a Svetlichny test. Thus, the for loop below iterates over all possible tests we produced 
### above (i.e., it produces as many Svetlichny tests as states in the list "states").

for test in circuits_for_backends:
        result = run_test(test, device, S, noise = "False")  
        print("Obtained vs ideal value:", result[0], result[1])

### (3.2) Noisy simulation.

For doing a noisy simulation, we need to first define a noise model. A simple one can be constructed by setting a noise parameter for each quantum gate used in the circuits. You can fix the noise prameters that suit your application.

In [ ]:
### Parameter values are chosen without any criteria (just for display).

### Define first an empty noise model.

noise_model = NoiseModel()

### Next, you can add noise to each possible quantum gate available in the Braket SDK.

noise_model.add_noise(Depolarizing(0.02), GateCriteria(Gate.H))
noise_model.add_noise(Depolarizing(0.0002), GateCriteria(Gate.T))
noise_model.add_noise(Depolarizing(0.0002), GateCriteria(Gate.V))
noise_model.add_noise(Depolarizing(0.0002), GateCriteria(Gate.Rx))
noise_model.add_noise(Depolarizing(0.0002), GateCriteria(Gate.Ry))
noise_model.add_noise(Depolarizing(0.0002), GateCriteria(Gate.Rz))
noise_model.add_noise(Depolarizing(0.0004), GateCriteria(Gate.CNot))
noise_model.add_noise(Depolarizing(0.0004), GateCriteria(Gate.CPhaseShift))
noise_model.add_noise(Depolarizing(0.0004), GateCriteria(Gate.CPhaseShift00))
noise_model.add_noise(Depolarizing(0.0004), GateCriteria(Gate.CPhaseShift01))
noise_model.add_noise(Depolarizing(0.0004), GateCriteria(Gate.CPhaseShift10))
noise_model.add_noise(BitFlip(0.0002), GateCriteria(Gate.H))
noise_model.add_noise(BitFlip(0.0002), GateCriteria(Gate.T))
noise_model.add_noise(BitFlip(0.0002), GateCriteria(Gate.V))
noise_model.add_noise(BitFlip(0.0002), GateCriteria(Gate.Rx))
noise_model.add_noise(BitFlip(0.0002), GateCriteria(Gate.Ry))
noise_model.add_noise(BitFlip(0.0002), GateCriteria(Gate.Rz))
noise_model.add_noise(BitFlip(0.0004), GateCriteria(Gate.CNot))
noise_model.add_noise(BitFlip(0.0004), GateCriteria(Gate.CPhaseShift))
noise_model.add_noise(BitFlip(0.0004), GateCriteria(Gate.CPhaseShift00))
noise_model.add_noise(BitFlip(0.0004), GateCriteria(Gate.CPhaseShift01))
noise_model.add_noise(BitFlip(0.0004), GateCriteria(Gate.CPhaseShift10))

We now run our list in a noisy device using the noise model defined above. 
As a result of noise, the obtained values differe from the ideal ones. Notice
that the quality of the test depends also on the number of shots.

In [ ]:
### Each element x of the list of circuits contains the information for testing the Svetlichny (or Mermin) inequality
### in a QPU or simulator.

S = 10000

results_ideal = []

device = LocalSimulator(backend="braket_dm")

for test in circuits_for_backends:
        result = run_test(test, device, S, noise = "True")  
        print("Obtained vs ideal value:", result[0], result[1])

### (3.3) Run a test on a QPU

Now that you have played with some simulations, you are ready to run a Svetlichny (or Mermin) test on an actual QPU. Let's give some indications on how to do that. The procedure is completly analogous to that of running your circuits on a local simulator. The philosophy is to replace the "device" we used above by an actual QPU or a simulator on the cloud. Also, you need to take care of the login credentials and, in case you are using it, setup the command line interphase.

In [ ]:
### Import some preliminary stuff.

from braket.aws import AwsDevice, AwsQuantumTask
from braket.circuits import Circuit, Gate, Noise, Observable, Instruction
import boto3

aws_account_id = boto3.client("sts").get_caller_identity()["Account"]
print(aws_account_id) 

You must define a device (it could be an actual QPU or one of AWS Braket simulators on the cloud). 

In [ ]:
device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1")

### Notice that there are several alternatives in Amazon Braket, for example:

# device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3")

Specify where your results are going to be stored:

In [ ]:
my_bucket = "the_name_you_want" # Choose the name of the bucket
my_prefix = "IonQ" # The name of the folder in the bucket
s3_location = (my_bucket, my_prefix)

Let us now illustrate how to run the test associated to the first state of the list "states". The information is contained in the first dictionary of the list "circuits_for_backends":

In [ ]:
### Extract the dictionary:

test_0 = circuits_for_backends[0]

### Out of the dictionary, extract the circuits associated to the Svetlichny test and save them in a list "test_list":

test_list = []

for i in range(len(test_0['Signed_circuits'])):
            circuit = test['Signed_circuits'][i][0].copy()
            test_list.append(circuit)

Finally, run the circuits contained in "test_list" on the device you defined above:

In [ ]:
### Specify the number of shots:

S = 1000

### Run

device.run_batch(test_list, s3_location, shots=S)